In [73]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt
import seaborn as sns

device = 'cuda' if torch.cuda.is_available() else 'cpu'

companies = ['AFKS', 'AFLT', 'AGRO', 'ALRS', 'CBOM', 'CHMF', 'ENPG', 'FEES', 'FIVE',
       'FIXP', 'GAZP', 'GLTR', 'GMKN', 'HYDR', 'IRAO', 'LKOH', 'MAGN', 'MGNT',
       'MOEX', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PHOR', 'PIKK', 'PLZL', 'POLY',
       'ROSN', 'RTKM', 'RUAL', 'SBER', 'SBERP', 'SGZH', 'SNGS', 'SNGSP',
       'TATN', 'TATNP', 'TCSG', 'TRNFP', 'VKCO', 'VTBR', 'YNDX']


def hour2time(hour):
    if hour == 1:
        res = '11:00:00'
    elif hour == 2:
        res = '12:00:00'
    elif hour == 3:
        res = '13:00:00'
    elif hour == 4:
        res = '14:00:00'
    elif hour == 5:
        res = '15:00:00'
    elif hour == 6:
        res = '16:00:00'
    elif hour == 7:
        res = '17:00:00'
    elif hour == 8:
        res = '18:00:00'
    elif hour == 9:
        res = '18:45:00'
    return res

# Closes

In [70]:
df = pd.read_csv('daily_backtest_data(NEW)').rename(columns={'Unnamed: 0':'date'})
df.head(1)

,date,Asset,Pred,Fact,date,hour,pct_pred,pct_fact
0,2023-03-06,GAZP,329.364395,349.6,2023-03-06,1,0.0,0.0


In [71]:
dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

In [72]:
bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

In [73]:
for i in range(0,len(bad_days)):    
    df = df[df.date != bad_days[i]]

In [74]:
df.date

Series([], Name: date, dtype: object)

In [75]:
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )
df.drop(['date','hour','pct_pred','pct_fact','Pred'],1,inplace=True)

/tmp/ipykernel_20905/3486096165.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_pred','pct_fact','Pred'],1,inplace=True)


In [76]:
pivot_df = df.pivot(index='Date', columns='Asset', values='Fact')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

In [77]:
pivot_df.to_csv('daily_cvxportfolio_fact_closes')

# Open

In [106]:
df = pd.read_csv('daily_additional_info')


#df = df[df['date']>= '2023-02-03']
df = df[df['date']>= '2023-03-14']
df = df[df['date']<= '2023-10-24']

df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )

df.drop(['date','hour','close','tradedate'],1,inplace=True)

pivot_df = df.pivot(index='Date', columns='secid', values='open')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

/tmp/ipykernel_6577/2617533290.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','close','tradedate'],1,inplace=True)


In [107]:
pivot_df.to_csv('daily_cvxportfolio_fact_opens')

In [108]:
pivot_df

,AFKS,AFLT,AGRO,ALRS,CBOM,CHMF,ENPG,FEES,FIVE,FIXP,...,SGZH,SNGS,SNGSP,TATN,TATNP,TCSG,TRNFP,VKCO,VTBR,YNDX
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-14 11:00:00,158.89,158.89,158.89,158.89,158.89,158.89,158.89,158.89,158.89,158.89,...,158.89,158.89,158.89,158.89,158.89,158.89,158.89,158.89,158.89,158.89
2023-03-15 11:00:00,161.50,161.50,161.50,161.50,161.50,161.50,161.50,161.50,161.50,161.50,...,161.50,161.50,161.50,161.50,161.50,161.50,161.50,161.50,161.50,161.50
2023-03-16 11:00:00,159.81,159.81,159.81,159.81,159.81,159.81,159.81,159.81,159.81,159.81,...,159.81,159.81,159.81,159.81,159.81,159.81,159.81,159.81,159.81,159.81
2023-03-17 11:00:00,160.19,160.19,160.19,160.19,160.19,160.19,160.19,160.19,160.19,160.19,...,160.19,160.19,160.19,160.19,160.19,160.19,160.19,160.19,160.19,160.19
2023-03-20 11:00:00,164.34,164.34,164.34,164.34,164.34,164.34,164.34,164.34,164.34,164.34,...,164.34,164.34,164.34,164.34,164.34,164.34,164.34,164.34,164.34,164.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-18 11:00:00,172.68,172.68,172.68,172.68,172.68,172.68,172.68,172.68,172.68,172.68,...,172.68,172.68,172.68,172.68,172.68,172.68,172.68,172.68,172.68,172.68
2023-10-19 11:00:00,169.95,169.95,169.95,169.95,169.95,169.95,169.95,169.95,169.95,169.95,...,169.95,169.95,169.95,169.95,169.95,169.95,169.95,169.95,169.95,169.95
2023-10-20 11:00:00,170.38,170.38,170.38,170.38,170.38,170.38,170.38,170.38,170.38,170.38,...,170.38,170.38,170.38,170.38,170.38,170.38,170.38,170.38,170.38,170.38


# Pred Returns

In [109]:
df = pd.read_csv('daily_backtest_data(NEW)').rename(columns={'Unnamed: 0':'date'})

dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

df = df[df['date']>= '2023-02-03']
df = df[df['date']<= '2023-10-24']

#bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

#for i in range(0,len(bad_days)):    
#    df = df[df.date != bad_days[i]]
    
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )

df.drop(['date','hour','pct_fact','Pred','Fact'],1,inplace=True)

pivot_df = df.pivot(index='Date', columns='Asset', values='pct_pred')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

/tmp/ipykernel_6577/1416836097.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_fact','Pred','Fact'],1,inplace=True)


In [111]:
pivot_df.to_csv('daily_cvxportfolio_pred_returns')

# Fact Returns

In [112]:
df = pd.read_csv('daily_backtest_data(NEW)').rename(columns={'Unnamed: 0':'date'})

dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

df = df[df['date']>= '2023-02-03']
df = df[df['date']<= '2023-10-24']

#bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

#for i in range(0,len(bad_days)):    
#    df = df[df.date != bad_days[i]]
    
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )

df.drop(['date','hour','pct_pred','Pred','Fact'],1,inplace=True)

pivot_df = df.pivot(index='Date', columns='Asset', values='pct_fact')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

/tmp/ipykernel_6577/2895406338.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_pred','Pred','Fact'],1,inplace=True)


In [114]:
pivot_df.to_csv('daily_cvxportfolio_fact_returns')